In [ ]:
#Use Augmentor to process images that are stored in memory, and are not read locally from a directory on a hard disk.
import Augmentor

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

(x_train, y_train), (x_test, y_test) = mnist.load_data()
#Because we are going to feed the network categorical data, we should convert y_train and y_test:
y_train = Augmentor.Pipeline.categorical_labels(y_train)
y_test = Augmentor.Pipeline.categorical_labels(y_test)

num_images, width, height = np.shape(x_train)
print("The matrix x_train contains %s images, with dimenions of %s x %s." % (num_images, width, height))
np.shape(x_train[0])
plt.imshow(x_train[0], cmap="Greys");

p = Augmentor.Pipeline()
#p.status()
p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
#p.status()
num_classes = 10
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
batch_size = 128
g = p.keras_generator_from_array(x_train, y_train, batch_size=10)
X, y = next(g)
plt.imshow(X[0].reshape(28,28), cmap="Greys")
print("The image above has the label %s." % int(np.nonzero(y[0])[0]))
#fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
#fit(self, x, y, batch_size=32, epochs=10, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)

h = model.fit_generator(g, steps_per_epoch=len(x_train)/batch_size, epochs=10, verbose=1)

In [ ]:
#Reading images from a local directory, augmenting them at run-time, and using a generator to pass the augmented stream of images to a Keras convolutional neural network
import Augmentor

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
num_classes = 10
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
#model.summary()

#!wget https://rawgit.com/myleott/mnist_png/master/mnist_png.tar.gz
#!tar -xf mnist_png.tar.gz

p = Augmentor.Pipeline("mnist_png/training")

p.flip_top_bottom(probability=0.1)
p.rotate(probability=0.3, max_left_rotation=5, max_right_rotation=5)

#p.status()

batch_size = 128
g = p.keras_generator(batch_size=batch_size)
images, labels = next(g)
print(labels[0])
plt.imshow(images[0].reshape(28, 28), cmap="Greys");

#fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
#fit(self, x, y, batch_size=32, epochs=10, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)
h = model.fit_generator(g, steps_per_epoch=len(p.augmentor_images)/batch_size, epochs=5, verbose=1)